In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Load the dataset
df = pd.read_csv('final_project_cleaned.csv')

2023-03-24 22:28:13.025385: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-24 22:28:13.060674: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-24 22:28:13.061255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 22:28:13.781039: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import tensorflow as tf
from tensorflow import keras
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [15]:
df

,text,label
0,Can I automatically top-up when traveling?,top_up_queries_or_issues
1,What kind of fiat currency can I used for hold...,other
2,I did not get the item I ordered. How should ...,other
3,Freeze my account it's been hacked.,needs_troubleshooting
4,is there a reason that my payment didnt go thr...,other
...,...,...
13079,Can you tell me what the disposable cards are ...,card_queries_or_issues
13080,The ATM won't give me my card back. I need it ...,card_queries_or_issues
13081,Can you please tell me why my card payments ar...,card_queries_or_issues
13082,The rate for a currency exchange was wrong whe...,needs_troubleshooting


In [16]:
df.reset_index(drop=True)

,text,label
0,Can I automatically top-up when traveling?,top_up_queries_or_issues
1,What kind of fiat currency can I used for hold...,other
2,I did not get the item I ordered. How should ...,other
3,Freeze my account it's been hacked.,needs_troubleshooting
4,is there a reason that my payment didnt go thr...,other
...,...,...
13079,Can you tell me what the disposable cards are ...,card_queries_or_issues
13080,The ATM won't give me my card back. I need it ...,card_queries_or_issues
13081,Can you please tell me why my card payments ar...,card_queries_or_issues
13082,The rate for a currency exchange was wrong whe...,needs_troubleshooting


In [ ]:
df

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.sparse import reorder

# Load the dataset
df = pd.read_csv('final_project_cleaned.csv')
df.reset_index(drop=True)

# Preprocess the text data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into text
    text = ' '.join(tokens)
    return text

df['text'] = df['text'].apply(preprocess_text)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Perform oversampling on the training set
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train.to_numpy().reshape(-1, 1), y_train)                                

# Define the RNN model using Keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

def create_rnn_model(num_words, max_len):
    model_rnn = Sequential()
    model_rnn.add(Embedding(input_dim=num_words, output_dim=128, input_length=max_len))
    model_rnn.add(SimpleRNN(units=64))
    model_rnn.add(Dense(units=64, activation='relu'))
    model_rnn.add(Dense(units=len(df['label'].unique()), activation='softmax'))
    model_rnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_rnn

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_resampled.ravel())
X_train_seq = tokenizer.texts_to_sequences(X_train_resampled.ravel())
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to same length
max_len = max(len(seq) for seq in X_train_seq)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# Define the pipeline
tfidf = TfidfVectorizer()
clf = KerasClassifier(build_fn=create_rnn_model, verbose=0)
pipe = Pipeline([('tfidf', tfidf), ('clf', clf)])
params = {
'tfidf__max_features': [5000, 10000, 20000],
'tfidf__ngram_range': [(1, 1), (1, 2)],
'clf__num_words': [5000, 10000, 20000],
'clf__max_len': [50, 100, 200],
'clf__epochs': [5, 10],
'clf__batch_size': [32, 64]
}


grid_search = GridSearchCV(pipe, param_grid=params, cv=3, verbose=1)
grid_search.fit(X_train_resampled.ravel(), y_train_resampled)


print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)
print("Test Score: ", grid_search.score(X_test, y_test))
print(classification_report(y_test, grid_search.predict(X_test)))



/tmp/ipykernel_9523/3061242157.py:75: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf = KerasClassifier(build_fn=create_rnn_model, verbose=0)
2023-03-24 22:02:45.428402: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,1196] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.




Fitting 3 folds for each of 216 candidates, totalling 648 fits


2023-03-24 22:02:45.561280: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,166] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:02:45.680485: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,201] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:02:45.864598: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,1953] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:02:46.046215: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_A

2023-03-24 22:02:50.408575: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,166] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:02:50.577601: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,201] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:02:50.753572: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,5526] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:02:50.957069: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_A

2023-03-24 22:02:54.941186: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,1196] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:02:55.059243: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,166] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:02:55.181177: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,201] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:02:55.338489: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_A

...

2023-03-24 22:04:11.036441: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[2] = [0,7745] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:11.146496: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,1196] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:11.259529: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,166] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:11.378423: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_

2023-03-24 22:04:16.167331: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,447] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:16.288252: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,1196] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:16.420073: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,166] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:16.580108: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_A

2023-03-24 22:04:20.876457: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[2] = [0,16444] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:20.980652: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,1196] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:21.086467: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,166] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:21.195446: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID

2023-03-24 22:04:25.074947: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[2] = [0,7745] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:25.217868: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,1196] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:25.327048: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[1] = [0,166] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.


2023-03-24 22:04:25.435950: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_

ValueError: 
All the 648 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
108 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ajay/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ajay/.local/lib/python3.8/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/ajay/.local/lib/python3.8/site-packages/keras/wrappers/scikit_learn.py", line 248, in fit
    return super().fit(x, y, **kwargs)
  File "/home/ajay/.local/lib/python3.8/site-packages/keras/wrappers/scikit_learn.py", line 175, in fit
    history = self.model.fit(x, y, **fit_args)
  File "/home/ajay/.loca

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.sparse import reorder

# Load the dataset
df = pd.read_csv('final_project_cleaned.csv')
df.reset_index(drop=True)

# Preprocess the text data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into text
    text = ' '.join(tokens)
    return text

df['text'] = df['text'].apply(preprocess_text)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Perform oversampling on the training set
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train.to_numpy().reshape(-1, 1), y_train)                                

# Define the RNN model using Keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

def create_rnn_model(num_words, max_len):
    model_rnn = Sequential()
    model_rnn.add(Embedding(input_dim=num_words, output_dim=128, input_length=max_len))
    model_rnn.add(SimpleRNN(units=64))
    model_rnn.add(Dense(units=64, activation='relu'))
    model_rnn.add(Dense(units=len(df['label'].unique()), activation='softmax'))
    model_rnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_rnn

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_resampled.ravel())
X_train_seq = tokenizer.texts_to_sequences(X_train_resampled.ravel())
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to same length
max_len = max(len(seq) for seq in X_train_seq)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# Define the pipeline
tfidf = TfidfVectorizer()
clf = KerasClassifier(build_fn=create_rnn_model, verbose=0)
pipe = Pipeline([('tfidf', tfidf), ('clf', clf)])

params = {
'tfidf__max_features': [5000, 10000, 20000],
'tfidf__ngram_range': [(1, 1), (1, 2)],
'clf__num_words': [5000, 10000, 20000],
'clf__max_len': [50, 100, 200],
'clf__epochs': [5, 10],
'clf__batch_size': [32, 64]
}

grid_search = GridSearchCV(pipe, param_grid=params, cv=3, verbose=1)
def preprocess_sparse_data(text):
# Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into text
    text = ' '.join(tokens)
    # Convert text to a sparse matrix
    sparse_matrix = tfidf.transform([text])
    # Reorder the sparse matrix
    sparse_matrix = reorder(sparse_matrix)
    return sparse_matrix

X_train_resampled_sparse = [preprocess_sparse_data(text) for text in X_train_resampled.ravel()]
X_test_sparse = [preprocess_sparse_data(text) for text in X_test]

#Fit the GridSearchCV with the preprocessed sparse data
grid_search.fit(X_train_resampled_sparse, y_train_resampled)

#Print the results
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)
print("Test Score: ", grid_search.score(X_test_sparse, y_test))
print(classification_report(y_test, grid_search.predict(X_test_sparse)))


/tmp/ipykernel_9523/3966514777.py:75: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf = KerasClassifier(build_fn=create_rnn_model, verbose=0)


NotFittedError: The TF-IDF vectorizer is not fitted

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.sparse import reorder

# Load the dataset
df = pd.read_csv('final_project_cleaned.csv')
df.reset_index(drop=True)

# Preprocess the text data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into text
    text = ' '.join(tokens)
    return text

df['text'] = df['text'].apply(preprocess_text)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Perform oversampling on the training set
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train.to_numpy().reshape(-1, 1), y_train) 

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Set parameters for the RNN model
vocab_size = 10000
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
epochs = 10
batch_size = 64

# Tokenize the text data
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train_resampled.squeeze())
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(X_train_resampled.squeeze())
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Define the RNN model architecture
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(4, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_padded, y_train_resampled, epochs=epochs, batch_size=batch_size)

# Evaluate the model on the test set
test_sequences = tokenizer.texts_to_sequences(X_test.squeeze())
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
_, test_accuracy = model.evaluate(test_padded, y_test)

# Generate predictions for the test set
y_pred = model.predict(test_padded)

# Print classification report
print(classification_report(y_test, y_pred.argmax(axis=1)))


Epoch 1/10


2023-03-24 22:12:50.840532: W tensorflow/core/framework/op_kernel.cc:1807] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'Cast_19' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/ajay/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/ajay/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/ajay/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/ajay/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/asyncio/

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.sparse import reorder
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Load the dataset
df = pd.read_csv('final_project_cleaned.csv')
df.reset_index(drop=True)

# Preprocess the text data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into text
    text = ' '.join(tokens)
    return text

df['text'] = df['text'].apply(preprocess_text)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Perform oversampling on the training set
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train.to_numpy().reshape(-1, 1), y_train) 

# Set parameters for the RNN model
vocab_size = 10000
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
epochs = 10
batch_size = 64

# Tokenize the text data
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train_resampled.squeeze())
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(X_train_resampled.squeeze())
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Define the RNN model architecture
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(4, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_padded, y_train_resampled, epochs=epochs, batch_size=batch_size)

# Evaluate the model on the test set
test_sequences = tokenizer.texts_to_sequences(X_test.squeeze())
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
_, test_accuracy = model.evaluate(test_padded, y_test)

# Generate predictions for the test set
# Generate predictions for the test set
y_pred = model.predict(test_padded)

# Print classification report
print(classification_report(y_test, y_pred.argmax(axis=1)))


Epoch 1/10


2023-03-24 22:17:53.248716: W tensorflow/core/framework/op_kernel.cc:1807] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'Cast_19' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/ajay/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/ajay/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/ajay/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/ajay/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/asyncio/

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
df = pd.read_csv("final_project_cleaned.csv")
df.reset_index(drop=True)

# Preprocess the data
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])
labels = df['label'].values
text = df['text'].values

# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(text)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(text)

# Pad the sequences
max_length = max([len(s.split()) for s in text])
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# Split the data into training and testing sets
train_padded, test_padded, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=42)

# Resample the training set to address class imbalance
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
train_padded, y_train_resampled = ros.fit_resample(train_padded, y_train)

# Define the model architecture
embedding_dim = 100
num_filters = 64
filter_size = 5
dropout_rate = 0.5

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(num_filters, filter_size, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define hyperparameters
epochs = 5
batch_size = 64

# Train the model
model.fit(train_padded, y_train_resampled, epochs=epochs, batch_size=batch_size)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_padded, y_test, verbose=2)
print('Test accuracy:', test_acc)


Epoch 1/5


2023-03-24 22:28:20.364372: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


240/240 [==============================] - 4s 16ms/step - loss: -123.8490 - accuracy: 0.2495
Epoch 2/5
240/240 [==============================] - 4s 16ms/step - loss: -2228.3167 - accuracy: 0.2500
Epoch 3/5
240/240 [==============================] - 4s 17ms/step - loss: -10150.2910 - accuracy: 0.2499
Epoch 4/5
240/240 [==============================] - 4s 17ms/step - loss: -26848.0195 - accuracy: 0.2500
Epoch 5/5
240/240 [==============================] - 4s 17ms/step - loss: -55626.1680 - accuracy: 0.2499
82/82 - 0s - loss: -6.1018e+04 - accuracy: 0.3145 - 250ms/epoch - 3ms/step
Test accuracy: 0.3144822418689728


In [6]:
import tensorflow as tf
from tensorflow import keras
from kerastuner import RandomSearch

# Define the hyperparameters to tune
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size, embedding_dim))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define the search space
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir',
    project_name='helloworld')

# Perform the hyperparameter search
tuner.search(train_padded, y_train_resampled, epochs=20, validation_data=(test_padded, y_test))

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test set
loss, accuracy = best_model.evaluate(test_padded, y_test)
print('Test accuracy:', accuracy)

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates